### SQL-Based Analysis of Product Sales

In [1]:
import sqlite3
import pandas as pd 


In [2]:
con = sqlite3.connect("Chinook_Sqlite.sqlite") #conncetion
cursor = con.cursor()  #cursor Object

In [6]:
cursor.execute('SELECT name FROM sqlite_master WHERE type="table";')
table = cursor.fetchall()
table

[('Album',),
 ('Artist',),
 ('Customer',),
 ('Employee',),
 ('Genre',),
 ('Invoice',),
 ('InvoiceLine',),
 ('MediaType',),
 ('Playlist',),
 ('PlaylistTrack',),
 ('Track',)]

##  Answer key business questions: 
1) Top-selling products
2) Revenue per region
3) Monthly performance

### Top-selling products

In [75]:
track = """SELECT t.Name AS Track_Name, COUNT(il.Quantity) AS Total_Quantity_Sold 
        FROM InvoiceLine il JOIN Track t ON il.Trackid = t.Trackid 
        GROUP BY il.trackid 
        ORDER BY Total_Quantity_Sold DESC LIMIT 10;"""
track_prod = pd.read_sql_query(track,con)
print(track_prod)

                  Track_Name  Total_Quantity_Sold
0          Balls to the Wall                    2
1           Inject The Venom                    2
2                 Snowballed                    2
3                   Overdose                    2
4            Deuces Are Wild                    2
5             Not The Doctor                    2
6          Por Causa De Você                    2
7  Welcome Home (Sanitarium)                    2
8                  Snowblind                    2
9                 Cornucopia                    2


## Revenue per region

In [8]:
rev_region = """SELECT c.Country, SUM(i.Total) AS Total_Revenu_By_Region 
                FROM CUSTOMER c 
                JOIN Invoice i ON i.Customerid = c.Customerid 
                GROUP BY Country ORDER BY Total_Revenu_By_Region DESC """

revenue = pd.read_sql_query(rev_region,con)
revenue


,Country,Total_Revenu_By_Region
0,USA,523.06
1,Canada,303.96
2,France,195.10
3,Brazil,190.10
4,Germany,156.48
5,United Kingdom,112.86
6,Czech Republic,90.24
7,Portugal,77.24
8,India,75.26
9,Chile,46.62


##  Monthly performance

In [58]:
perf = '''Select strftime("%Y",i.InvoiceDate) AS Year, 
            strftime("%m",i.InvoiceDate) AS Month,
            Sum(il.UnitPrice * il.Quantity) AS  Monthly_Revenu 
            FROM InvoiceLine il 
            JOIN Invoice i ON i.InvoiceId = il.InvoiceId 
            GROUP BY Year,Month ORDER BY Year,Month;'''
month_num_map = {
    '01': 'January', '02': 'February', '03': 'March',
    '04': 'April', '05': 'May', '06': 'June',
    '07': 'July', '08': 'August', '09': 'September',
    '10': 'October', '11': 'November', '12': 'December'
    }
monthly_perf = pd.read_sql_query(perf,con)
monthly_perf["Month"]= monthly_perf["Month"].map(month_num_map)
monthly_perf

,Year,Month,Monthly_Revenu
0,2009,January,35.64
1,2009,February,37.62
2,2009,March,37.62
3,2009,April,37.62
4,2009,May,37.62
5,2009,June,37.62
6,2009,July,37.62
7,2009,August,37.62
8,2009,September,37.62
9,2009,October,37.62


### CustomerRevenue By using WindowFunction

In [73]:
query = """WITH CustomerRevenue AS (
    SELECT c.Country, c.CustomerId, 
    concat(c.FirstName," ",c.LastName) AS CustomerName,
    sum(il.UnitPrice * il.Quantity) AS TotalRevenue
    FROM Customer c
    JOIN Invoice i ON c.CustomerId = i.CustomerId
    JOIN InvoiceLine il ON i.InvoiceId = il.InvoiceId
    GROUP BY c.Country, c.CustomerId
),
RankedCustomers AS (
    SELECT *,
    RANK() OVER (PARTITION BY Country ORDER BY TotalRevenue DESC) AS Revenue_Rank,
    ROW_NUMBER() OVER (PARTITION BY Country ORDER BY TotalRevenue DESC) AS Row_Num
    FROM CustomerRevenue
)
SELECT *
FROM RankedCustomers
where Revenue_Rank=1 and Row_Num = 1
ORDER BY Country;

"""

window_func = pd.read_sql_query(query,con)
window_func

,Country,CustomerId,CustomerName,TotalRevenue,Revenue_Rank,Row_Num
0,Argentina,56,Diego Gutiérrez,37.62,1,1
1,Australia,55,Mark Taylor,37.62,1,1
2,Austria,7,Astrid Gruber,42.62,1,1
3,Belgium,8,Daan Peeters,37.62,1,1
4,Brazil,1,Luís Gonçalves,39.62,1,1
5,Canada,3,François Tremblay,39.62,1,1
6,Chile,57,Luis Rojas,46.62,1,1
7,Czech Republic,6,Helena Holý,49.62,1,1
8,Denmark,9,Kara Nielsen,37.62,1,1
9,Finland,44,Terhi Hämäläinen,41.62,1,1
